<a href="https://colab.research.google.com/github/JaimeLopezGarrido/Trabajos_Labo1/blob/main/Prueba_Op_Bay_T04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python

In [ ]:
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [ ]:
%%shell
mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1 || true

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle 2>/dev/null || true
chmod 600 ~/.kaggle 2>/dev/null || true

mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"

if [ ! -f "$archivo_destino_bucket" ]; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi

if [ ! -f "$archivo_destino" ]; then
  cp  $archivo_destino_bucket  $archivo_destino
fi

echo "Dataset listo en $archivo_destino"

Dataset listo en /content/datasets/dataset_pequeno.csv


# R

In [ ]:
# --- R: setup, instalación de paquetes (solo si falta) y configuraciones ---
# Reinicia kernel si instalas paquetes y luego corre de nuevo desde aquí.

# Instalación segura: solo instala si hace falta
pkg_check <- function(pkgs){
  for(p in pkgs){
    if(!requireNamespace(p, quietly = TRUE)){
      install.packages(p, repos = "https://cloud.r-project.org", dependencies = TRUE)
    }
    library(p, character.only = TRUE)
  }
}

# paquetes base que vamos a usar
base_pkgs <- c("data.table", "rpart", "parallel", "rlist")
pkg_check(base_pkgs)

# paquetes necesarios para mlrMBO
mbo_pkgs <- c("DiceKriging", "mlrMBO", "ParamHelpers", "mlr")
pkg_check(mbo_pkgs)



In [ ]:
# --- Parámetros globales del experimento ---
PARAM <- list()
PARAM$semilla_primigenia <- 100007   # tu semilla
PARAM$experimento <- "Prueba Op Bayesiana 3 Tarea 04"
# Modo de ejecución:
# TEST_MODE = TRUE  -> versiones rápidas para validar (minutos)
# TEST_MODE = FALSE -> ejecución completa (horas)
TEST_MODE <- TRUE

if (TEST_MODE) {
  PARAM$BO_iter <- 3   # iteraciones muy pocas para test
  # acotar espacio para que cada evaluación sea rápida
  PARAM$hs <- ParamHelpers::makeParamSet(
    ParamHelpers::makeNumericParam("cp", lower = -0.2, upper = 0.01),
    ParamHelpers::makeIntegerParam("minsplit", lower = 200L, upper = 600L),
    ParamHelpers::makeIntegerParam("minbucket", lower = 100L, upper = 300L),
    ParamHelpers::makeIntegerParam("maxdepth", lower = 4L, upper = 8L),
    ParamHelpers::makeIntegerParam("num_trees_max", lower = 8L, upper = 16L),
    ParamHelpers::makeNumericParam("feature_fraction", lower = 0.6, upper = 1.0),
    forbidden = quote(minbucket > 0.5 * minsplit)
  )
} else {
  PARAM$BO_iter <- 40  # ejecución completa
  PARAM$hs <- ParamHelpers::makeParamSet(
    ParamHelpers::makeNumericParam("cp", lower = -1, upper = 0.1),
    ParamHelpers::makeIntegerParam("minsplit", lower = 1L, upper = 8000L),
    ParamHelpers::makeIntegerParam("minbucket", lower = 1L, upper = 4000L),
    ParamHelpers::makeIntegerParam("maxdepth", lower = 3L, upper = 20L),
    ParamHelpers::makeIntegerParam("num_trees_max", lower = 8L, upper = 128L),
    ParamHelpers::makeNumericParam("feature_fraction", lower = 0.2, upper = 1.0),
    forbidden = quote(minbucket > 0.5 * minsplit)
  )
}


In [ ]:

# carpeta de trabajo
setwd("/content/buckets/b1/exp")
dir.create(PARAM$experimento, showWarnings = FALSE)
setwd(paste0("/content/buckets/b1/exp/", PARAM$experimento))

# --- funciones auxiliares (particionar, ArbolSimple, CV, etc.) ---
particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)
  bloque <- unlist(mapply(function(x, y) rep(y, x), division, seq(from = start, length.out = length(division))))
  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N], by = agrupa]
}

# Fun: entrena ensemble de rpart sobre dtrain y evalua en dtest. Devuelve ganancia (entera, no normalizada)
ArbolSimple <- function(fold_test, param_rpart, dataset_local, semilla_local) {
  t0 <- Sys.time()
  # separo train/test segun fold
  dtrain <- dataset_local[fold != fold_test, ]
  dtest  <- dataset_local[fold == fold_test, ]

  # prepara
  tb_prediccion <- dtest[, .(numero_de_cliente)]
  tb_prediccion[, prob_acumulada := 0]

  # parametros rpart locales (no tocan variables globales)
  rpart_ctrl <- list(
    cp = param_rpart$cp,
    minsplit = as.integer(param_rpart$minsplit),
    minbucket = as.integer(param_rpart$minbucket),
    maxdepth = as.integer(param_rpart$maxdepth)
  )

  num_trees <- as.integer(param_rpart$num_trees_max)
  feat_frac  <- as.numeric(param_rpart$feature_fraction)

  set.seed(as.integer(semilla_local) + fold_test)

  campos_buenos <- setdiff(colnames(dtrain), c("clase_ternaria", "numero_de_cliente", "fold"))

  for (arbolito in seq_len(num_trees)) {
    qty_campos_a_utilizar <- max(1, as.integer(length(campos_buenos) * feat_frac))
    campos_random <- sample(campos_buenos, qty_campos_a_utilizar)
    formulita <- as.formula(paste("clase_ternaria ~", paste(campos_random, collapse = " + ")))

    modelo <- rpart::rpart(formulita, data = dtrain, xval = 0, control = rpart_ctrl)
    prediccion <- predict(modelo, dtest, type = "prob")

    if (is.null(dim(prediccion))) {
      probs_baja2 <- rep(0, nrow(dtest))
    } else {
      probs_baja2 <- prediccion[, "BAJA+2"]
      probs_baja2[is.na(probs_baja2)] <- 0
    }
    tb_prediccion[, prob_acumulada := prob_acumulada + probs_baja2]
  }

  umbral_corte <- (1 / 40) * num_trees
  tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

  dtest2 <- copy(dtest)
  dtest2[, Predicted := tb_prediccion$Predicted]

  ganancia_testing <- dtest2[Predicted == 1, sum(ifelse(clase_ternaria == "BAJA+2", 117000, -3000))]
  if (is.na(ganancia_testing)) ganancia_testing <- 0

  t1 <- Sys.time()
  dur <- as.numeric(difftime(t1, t0, units = "secs"))
  message(sprintf("Fold %d: ganancia=%d (tiempo %.1f s)", fold_test, ganancia_testing, dur))
  return(ganancia_testing)
}

# CV en folds paralelos
ArbolesCrossValidation <- function(param_rpart, qfolds, pagrupa, semilla, dataset_local) {
  divi <- rep(1, qfolds)
  particionar(dataset_local, divi, seed = semilla, agrupa = pagrupa)

  # ejecutar folds de manera secuencial o paralela según TEST_MODE
  cores_avail <- parallel::detectCores(logical = FALSE)
  mc.cores <- ifelse(TEST_MODE, 1, max(1, min(cores_avail, 4))) # en test uso 1 para estabilidad en Colab
  message(sprintf("Iniciando CV con %d folds usando mc.cores=%d", qfolds, mc.cores))

  ganancias <- mcmapply(ArbolSimple,
                        seq(qfolds),
                        MoreArgs = list(param_rpart, dataset_local, semilla),
                        SIMPLIFY = FALSE,
                        mc.cores = mc.cores)

  dataset_local[, fold := NULL]
  ganancia_promedio <- mean(unlist(ganancias))
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds
  return(ganancia_promedio_normalizada)
}

# función objetivo para MBO: recibe lista x con hiperparámetros
EstimarGanancia <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1
  message(sprintf("ITERACION %d - %s", GLOBAL_iteracion, format(Sys.time(), "%Y-%m-%d %H:%M:%S")))

  # asegurar tipos
  x$minsplit <- as.integer(x$minsplit)
  x$minbucket <- as.integer(x$minbucket)
  x$maxdepth <- as.integer(x$maxdepth)
  x$num_trees_max <- as.integer(x$num_trees_max)
  x$feature_fraction <- as.numeric(x$feature_fraction)

  # Para test rapido: podes usar una muestra reducida del dataset
  if (TEST_MODE) {
    dataset_local <- dataset[sample(.N, min(.N, 10000))]  # usa hasta 10k filas para test
  } else {
    dataset_local <- dataset
  }

  ganancia <- ArbolesCrossValidation(param_rpart = x, qfolds = 3, pagrupa = "clase_ternaria", semilla = PARAM$semilla_primigenia, dataset_local = dataset_local)
  message(sprintf("ITERACION %d -> ganancia normalizada = %f", GLOBAL_iteracion, ganancia))
  return(ganancia)
}


In [ ]:

# --- lectura dataset (igual a tu flujo) ---
dataset <- data.table::fread("/content/datasets/dataset_pequeno.csv")

# Si querés chequear rápidamente tamaño
message(sprintf("Dataset cargado: %d filas, %d cols", nrow(dataset), ncol(dataset)))


In [ ]:

# --- logging y estado para retomar ---
archivo_log <- "BO_log.txt"
archivo_BO  <- "bayesian_rpart_ensemble.RDATA"

GLOBAL_iteracion <- 0
GLOBAL_mejor <- -Inf

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_mejor <- tabla_log[, max(ganancia)]
}

# --- Configuración mlrMBO ---
funcion_optimizar <- EstimarGanancia
mlr::configureMlr(show.learner.output = FALSE)

obj.fun <- mlr::makeSingleObjectiveFunction(
  fn = funcion_optimizar,
  minimize = FALSE,
  noisy = TRUE,
  par.set = PARAM$hs,
  has.simple.signature = FALSE
)

ctrl <- mlrMBO::makeMBOControl(save.on.disk.at.time = 600, save.file.path = archivo_BO)
ctrl <- mlrMBO::setMBOControlTermination(ctrl, iters = PARAM$BO_iter)
ctrl <- mlrMBO::setMBOControlInfill(ctrl, crit = mlrMBO::makeMBOInfillCritEI())

surr.km <- mlr::makeLearner("regr.km", predict.type = "se", covtype = "matern3_2", control = list(trace = TRUE))

# RUN or resume
if (!file.exists(archivo_BO)) {
  message("Iniciando MBO...")
  t_start_all <- Sys.time()
  bayesiana_salida <- mlrMBO::mbo(fun = obj.fun, learner = surr.km, control = ctrl)
  t_end_all <- Sys.time()
  message(sprintf("Tiempo total MBO: %.1f min", as.numeric(difftime(t_end_all, t_start_all, units = "mins"))))
} else {
  message("Continuando MBO existente...")
  bayesiana_salida <- mlrMBO::mboContinue(archivo_BO)
}

# guardar y extraer mejores
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y)
fwrite(tb_bayesiana, file = archivo_log, sep = "\t")

best_row <- tb_bayesiana[1]
PARAM$out <- list()
PARAM$out$rpart_ensemble_mejores_hiperparametros <- best_row[, .(cp, minsplit, minbucket, maxdepth, num_trees_max = num_trees_max, feature_fraction = feature_fraction)]

message("Mejores hiperparametros encontrados:")
print(PARAM$out$rpart_ensemble_mejores_hiperparametros)

# Fin
message("FIN - ", format(Sys.time(), "%Y-%m-%d %H:%M:%S"))